In [0]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("microdados_enem") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## Análise e resposta às perguntas levantadas

A medida de dispersão escolhida para entender o comportamento das notas foi a mediana (aqui calculada como o percentil 50%) de cada nota. Isto se deu porque no caso de notas, podem existir outliers que influenciariam significativamente outras medidas mais usadas, como a média.

### 1. Existem diferenças significativas nos desempenhos de estudantes de acordo com sua Unidade da Federação?

Inicialmente, a ideia era fazer a análise utilizando as informações de localização da escola de cada estudante, mas por este ser um campo com muitos valores nulos (quase 3M), foi utilizada a localização de realização da prova. 

Já que o local de prova é de escolha do candidato (não é imposto pelo exame), espera-se que, de forma geral, ele escolha um local próximo de sua residência. Como essa análise olha apenas para UF e não para município ou cidade, o local de prova parece uma boa aproximação.

In [0]:
df = spark.table("enem_raw_data")

null_counts = df.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])

display(null_counts.select('CO_MUNICIPIO_ESC', 'SG_UF_ESC', 'CO_MUNICIPIO_PROVA', 'SG_UF_PROVA'))

CO_MUNICIPIO_ESC,SG_UF_ESC,CO_MUNICIPIO_PROVA,SG_UF_PROVA
2975449,2975449,0,0


In [0]:
%sql
SELECT
    dlp.SG_UF_PROVA,
    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

FROM fato_resultado_enem fre
LEFT JOIN dim_local_prova dlp
    ON fre.ID_LOCAL_PROVA = dlp.ID_LOCAL_PROVA
GROUP BY dlp.SG_UF_PROVA
ORDER BY dlp.SG_UF_PROVA

SG_UF_PROVA,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
AC,24274,475.8,470.7
AL,82760,489.7,475.7
AM,92916,462.7,465.5
AP,28807,463.4,470.4
BA,324268,488.8,480.4
CE,241960,506.2,477.5
DF,72975,548.7,513.5
ES,73724,555.5,509.3
GO,149110,523.4,494.5
MA,165756,460.6,461.3


Em relação às notas de matemática, a unidade da federação com melhor desempenho médio foi SP, seguida por MG, SC, RS e ES. As UFs com pior desempenho foram MA, AM, AP, PA e AC.

Já em relação à Ciências da Natureza, a UF melhor classificada foi SC, seguida por MG, SP, DF e RS. A unidade com pior classificação também foi MA, seguida por AM, PA, PI e AP. 

### 2. Existem diferenças significativas nos desempenhos de estudantes do sexo masculino e feminino?

In [0]:
%sql
SELECT
    dp.TP_SEXO,
    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

FROM fato_resultado_enem fre
LEFT JOIN dim_participante dp
    ON fre.ID_PARTICIPANTE = dp.ID_PARTICIPANTE
GROUP BY dp.TP_SEXO
ORDER BY dp.TP_SEXO

TP_SEXO,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
F,2411185,503.1,483.7
M,1522770,561.1,512.2


É possível ver que tanto para matemática quanto para ciências da natureza, os estudantes do sexo masculino apresentam desempenho superior às estudantes do sexo feminino de forma geral.

### 3. Existem diferenças significativas nos desempenhos de estudantes de acordo com sua cor/raça/etnia?

In [0]:
%sql
SELECT
    CASE dp.TP_COR_RACA
        WHEN 0 THEN 'Não declarado'
        WHEN 1 THEN 'Branca'
        WHEN 2 THEN 'Preta'
        WHEN 3 THEN 'Parda'
        WHEN 4 THEN 'Amarela'
        WHEN 5 THEN 'Indígena'
        ELSE 'Desconhecido'
    END AS raca_cor,

    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

FROM fato_resultado_enem fre
LEFT JOIN dim_participante dp
    ON fre.ID_PARTICIPANTE = dp.ID_PARTICIPANTE
GROUP BY dp.TP_COR_RACA
ORDER BY dp.TP_COR_RACA

raca_cor,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
Não declarado,52575,523.0,498.9
Branca,1575848,574.2,517.5
Preta,509511,478.6,473.9
Parda,1706798,492.5,478.1
Amarela,64512,517.1,489.9
Indígena,24711,442.0,451.0


De acordo com a visão, estudantes com etnia "Branca" são os que possuem melhor desempenho médio dentre todos. Em seguida estão os sem declaração (raça/cor = Não declarado) e então raça/cor "Amarela", "Parda", "Preta" e "Indígena".

A quantidade de participantes com declaração "Amarela", "Indígena" e "Não declarado" é significativamente menor que as demais (1.64%, 0.63% e 1.34% do total, respectivamente) o que dificulta a comparação dos resultados. Seria interessante repetir a análise em outros anos ou combinar com outras análises similares

###4. Existem diferenças significativas nos desempenhos de estudantes de acordo com o tipo da escola que frenquentaram?

In [0]:
%sql
SELECT
    CASE dp.TP_ESCOLA
        WHEN 1 THEN 'Não respondeu'
        WHEN 2 THEN 'Pública'
        WHEN 3 THEN 'Privada'
        ELSE 'Não informado'
    END AS tipo_escola,

    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

FROM fato_resultado_enem fre
LEFT JOIN dim_participante dp
    ON fre.ID_PARTICIPANTE = dp.ID_PARTICIPANTE
GROUP BY dp.TP_ESCOLA
ORDER BY dp.TP_ESCOLA

tipo_escola,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
Não respondeu,2532796,526.0,498.4
Pública,1166540,492.4,473.2
Privada,234619,646.6,552.5


Neste caso, os estudantes de escola privada apresentam melhor desempenho, seguidos pelos estudantes cujo tipo de escola não foi identificado e por último, os estudantes de escola pública.

É interessante oservar que para a imensa maioria dos respondentes (64.38%) não foi possível determinar o tipo de escola em que estudaram. Sendo assim, ainda que seja possível comparar as notas dos três grupos, não parecem amostras representativas da população e o melhor seria desconsiderar a análise por enquanto e investir tempo em técnicas para melhorar a qualidade dos dados.

### 5. Existem diferenças significativas nos desempenhos de estudantes de acordo com o nível de escolaridade que seus pais recebram?

#### Análise de acordo com o nível escolar da mãe

In [0]:
%sql
    SELECT
    CASE ds.Q002
        WHEN 'A' THEN 'Nunca estudou.'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do EF.'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano do EF, mas não a 8ª série/9º ano.'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do EF, mas não o ensino médio.'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        WHEN 'H' THEN 'Não sei.'
        ELSE 'Não informado'
    END AS escolaridade_mae,

    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

    FROM fato_resultado_enem fre
    LEFT JOIN dim_socioeconomico ds
        ON fre.ID_SOCIOECONOMICO = ds.ID_SOCIOECONOMICO
    GROUP BY ds.Q002
    ORDER BY ds.Q002

escolaridade_mae,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
Nunca estudou.,110831,425.3,454.5
Não completou a 4ª série/5º ano do EF.,505525,455.4,465.3
"Completou a 4ª série/5º ano do EF, mas não a 8ª série/9º ano.",437022,476.6,471.7
"Completou a 8ª série/9º ano do EF, mas não o ensino médio.",473854,490.3,476.0
"Completou o Ensino Médio, mas não completou a Faculdade.",1377605,525.6,493.3
"Completou a Faculdade, mas não completou a Pós-graduação.",455268,603.9,531.3
Completou a Pós-graduação.,441893,622.5,540.6
Não sei.,131957,463.1,462.5


Avaliando as notas de acordo com o nível de escolaridade da mãe, percebe-se que o grupo com melhor desempenho é o com mães que completaram a pós graduação (maior nível de escolaridade disponível dentre as opções), seguidos pelas que completaram a faculdade, o ensino médio, o ensino fundamental e assim por diante.

Esta distribuição faz sentido pois pais com maior nível de escolaridade tendem a valorizar mais o desempenho escolar de seus filhos, já que eles também se preocupam com a própria educação.

#### Análise de acordo com o nível escolar do pai

In [0]:
%sql
    SELECT
    CASE ds.Q001
        WHEN 'A' THEN 'Nunca estudou'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do EF'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano do EF, mas não a 8ª série/9º ano'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do EF, mas não o ensino médio'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        WHEN 'H' THEN 'Não sei.'
        ELSE 'Não informado'
    END AS escolaridade_pai,

    COUNT(*) AS qtd_participantes,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_MT, 0.5), 2) AS mediana_matematica,
    ROUND(PERCENTILE_APPROX(fre.NU_NOTA_CN, 0.5), 2) AS mediana_ciencias_natureza

    FROM fato_resultado_enem fre
    LEFT JOIN dim_socioeconomico ds
        ON fre.ID_SOCIOECONOMICO = ds.ID_SOCIOECONOMICO
    GROUP BY ds.Q001
    ORDER BY ds.Q001

escolaridade_pai,qtd_participantes,mediana_matematica,mediana_ciencias_natureza
Nunca estudou,174712,435.3,456.1
Não completou a 4ª série/5º ano do EF,700175,467.0,469.0
"Completou a 4ª série/5º ano do EF, mas não a 8ª série/9º ano",513213,493.8,478.8
"Completou a 8ª série/9º ano do EF, mas não o ensino médio",438139,508.0,484.2
"Completou o Ensino Médio, mas não completou a Faculdade.",1114164,541.8,501.2
"Completou a Faculdade, mas não completou a Pós-graduação.",333981,627.4,543.4
Completou a Pós-graduação.,256045,650.7,555.3
Não sei.,403526,482.8,473.4


Em relação à escolaridade do pai, vê-se a mesma relação de quanto maior a escolaridade, maior o desempenho do filho. 

### Autoavaliação

De forma geral, os objetivos definidos no início do projeto foram atendidos já que as perguntas propostas foram respondidas. É importante lembrar que o projeto teve como base os microdados do ENEM de 2023 e os comportamentos observados podem não ser recorrentes em outros anos.


Pensando em evoluções futuras, seria interessante reunir microdados do ENEM de anos anteriores e refazer essas análises para entender se os comportamentos observados são ou não pontuais e se precisam de políticas públicas específicas que os envolvam.